<a href="https://colab.research.google.com/github/Far-ch/Signals-and-Systems-Project/blob/main/Signal_and_Systems_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#YOLO
from ultralytics import YOLO
import cv2, torch, numpy as np, matplotlib.pyplot as plt

VIDEO_IN  = "person1.mp4"
SNAPSHOT  = "first_detected.jpg"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MODEL  = "yolov8m.pt"
model  = YOLO(MODEL).to(DEVICE)
CLASSES = {
    0: "person",1: "bicycle",2: "car",3: "motorcycle",4: "airplane",5: "bus",6: "train",7: "truck",8: "boat",9: "traffic light",10: "fire hydrant",
11: "stop sign", 12: "parking meter",13: "bench",14: "bird",15: "cat",16: "dog",17: "horse",18: "sheep",19: "cow",20: "elephant",21: "bear",
22: "zebra",23: "giraffe",24: "backpack",25: "umbrella",26: "handbag",27: "tie",28: "suitcase",29: "frisbee",30: "skis",31: "snowboard",32: "sports ball",
33: "kite",34: "baseball bat",35: "baseball glove",36: "skateboard",37: "surfboard",38: "tennis racket",39: "bottle",40: "wine glass",41: "cup",
42: "fork",43: "knife",44: "spoon",45: "bowl",46: "banana",47: "apple",48: "sandwich",49: "orange",50: "broccoli",51: "carrot",
52: "hot dog", 53: "pizza",54: "donut",55: "cake",56: "chair",57: "couch",58: "potted plant",59: "bed",60: "dining table",61: "toilet",
62: "tv",63: "laptop",64: "mouse",65: "remote",66: "keyboard",67: "cell phone",68: "microwave",69: "oven",70: "toaster",71: "sink",72: "refrigerator",
73: "book",74: "clock",75: "vase",76: "scissors",77: "teddy bear",78: "hair drier",79: "toothbrush"}

CLASS_IDS = list(CLASSES.keys())
rng = np.random.default_rng(42)
COLORS = {cls_id: tuple(rng.integers(64, 256, 3).tolist()) for cls_id in CLASS_IDS}

BOX_THICK = 3
FONT_SCALE = 1.0
FONT_THICK = 2


def detect(img_rgb, imgsz=960, conf=0.3):
    """Detect objects and return list of boxes: (x1, y1, x2, y2, class_id, conf)"""
    res = model.predict(img_rgb, imgsz=imgsz, conf=conf,
                        classes=CLASS_IDS, device=DEVICE,
                        verbose=False)[0]
    out = []
    for b in res.boxes:
        out.append((*map(int, b.xyxy[0]), int(b.cls[0]), float(b.conf[0])))
    return out


def robust_detect(frame_bgr):
    H, W = frame_bgr.shape[:2]

    #Full-frame
    boxes = detect(cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB), 1280, 0.3)
    if boxes:
        return boxes

    #Center crop + upscale
    cw, ch = int(W * 0.6), int(H * 0.7)
    x0, y0 = (W - cw) // 2, (H - ch) // 2
    crop = frame_bgr[y0:y0+ch, x0:x0+cw]
    crop_up = cv2.resize(crop, (1280, 1280))
    boxes_crop = detect(cv2.cvtColor(crop_up, cv2.COLOR_BGR2RGB), 1280, 0.25)
    if boxes_crop:
        sx, sy = cw / 1280, ch / 1280
        return [(int(x1*sx + x0), int(y1*sy + y0),
                 int(x2*sx + x0), int(y2*sy + y0), cls, conf)
                for x1, y1, x2, y2, cls, conf in boxes_crop]

    #Sliding tiles
    out = []
    tile, stride = 640, 320
    for y in range(0, H, stride):
        for x in range(0, W, stride):
            tile_bgr = frame_bgr[y:y+tile, x:x+tile]
            boxes_tile = detect(cv2.cvtColor(tile_bgr, cv2.COLOR_BGR2RGB), 640, 0.25)
            for x1, y1, x2, y2, cls, conf in boxes_tile:
                out.append((x1+x, y1+y, x2+x, y2+y, cls, conf))
    return out


cap = cv2.VideoCapture(VIDEO_IN)
ok, frame = cap.read()
cap.release()
assert ok, f" Couldn't read first frame of {VIDEO_IN}"

boxes = robust_detect(frame.copy())
for x1, y1, x2, y2, cls, conf in boxes:
    label_name = CLASSES.get(cls, "unknown")
    label_text = f"{label_name} {conf:.2f}"

    color = COLORS.get(cls, (0, 255, 255))
    cv2.rectangle(frame, (x1, y1), (x2, y2), color, BOX_THICK)

    (tw, th), _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX,
                                  FONT_SCALE, FONT_THICK)
    cv2.rectangle(frame, (x1, y1 - th - 8), (x1 + tw, y1), color, -1)
    cv2.putText(frame, label_text, (x1, y1 - 5),
                cv2.FONT_HERSHEY_SIMPLEX, FONT_SCALE,
                (255, 255, 255), FONT_THICK, cv2.LINE_AA)


cv2.imwrite(SNAPSHOT, frame)
plt.figure(figsize=(12,6))
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

In [ ]:
#Fast-R-CNN
!pip -q install --upgrade torch torchvision torchaudio
import cv2, torch, matplotlib.pyplot as plt
import numpy as np
from torchvision import models, transforms

# COCO classes
CLASSES = {
    0:"background", 1:"person", 2:"bicycle", 3:"car", 4:"motorcycle", 5:"airplane",
    6:"bus",7:"train",8:"truck",9:"boat",10:"trafficlight",11:"firehydrant",
    13:"bench",14:"bird",15:"cat",16:"dog",17:"horse",18:"sheep",19:"cow",
    20:"elephant",21:"bear",22:"zebra",23:"giraffe",24:"backpack",25:"umbrella",
    27:"handbag",28:"tie",31:"snowboard",32:"sportsball",33:"kite",34:"baseballbat",
    35:"baseballglove",36:"skateboard",37:"surfboard",38:"tennisracket",39:"bottle",
    40:"wineglass",41:"cup",42:"fork",43:"knife",44:"spoon",45:"bowl",46:"banana",
    47:"apple",48:"sandwich",49:"orange",50:"broccoli",51:"carrot",52:"hotdog",
    53:"pizza",54:"donut",55:"cake",56:"chair",57:"couch",58:"pottedplant",59:"bed",
    60:"diningtable",61:"toilet",62:"tv",63:"laptop",64:"mouse",65:"remote",
    66:"keyboard",67:"cellphone",68:"microwave",69:"oven",70:"toaster",71:"sink",
    72:"refrigerator",73:"book",74:"clock",75:"vase",76:"scissors",77:"teddybear",
    78:"hairdrier",79:"toothbrush"
}

VIDEO_PATH = "person1.mp4"
cap = cv2.VideoCapture(VIDEO_PATH)
ok, frame_bgr = cap.read()
cap.release()
assert ok, "Could not read first frame"

frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)

device = "cuda" if torch.cuda.is_available() else "cpu"
model  = models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
model.to(device).eval()

transform = transforms.Compose([transforms.ToTensor()])   # converts to [0,1] + CHW
img_tensor = transform(frame_rgb).to(device)


with torch.no_grad():
    preds = model([img_tensor])[0]   # dict with boxes, labels, scores

boxes   = preds["boxes"].cpu().numpy()
labels  = preds["labels"].cpu().numpy()
scores  = preds["scores"].cpu().numpy()

CONF_TH = 0.50
keep = scores >= CONF_TH

boxes, labels, scores = boxes[keep], labels[keep], scores[keep]

OUT = frame_bgr.copy()
for (x1,y1,x2,y2), cls_id, conf in zip(boxes, labels, scores):
    cls_id = int(cls_id)
    name   = CLASSES.get(cls_id, f"id{cls_id}")
    label  = f"{name} {conf:.2f}"

    color = (0,255,0)   # green boxes; change if you like
    cv2.rectangle(OUT, (int(x1),int(y1)), (int(x2),int(y2)), color, 2)
    (tw, th), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
    cv2.rectangle(OUT, (int(x1), int(y1)-th-8), (int(x1)+tw, int(y1)), color, -1)
    cv2.putText(OUT, label, (int(x1), int(y1)-4),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,0), 2)

cv2.imwrite("fast_rcnn_detect.jpg", OUT[:, :, ::-1])  # BGR→RGB file

plt.figure(figsize=(12,6))
plt.imshow(cv2.cvtColor(OUT, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title("Fast-R-CNN detection ")
plt.show()


In [ ]:
#KCF
from ultralytics import YOLO
import cv2
import time

model = YOLO('yolov8n.pt')

cap = cv2.VideoCapture('person4.mp4')
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps_input = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, fps_input, (width, height))

trackers = []
labels = []
frame_count = 0
yolo_ran = False  # برای کنترل اینکه YOLO فقط یک بار اجرا شود

total_start = time.perf_counter()
window_start = total_start
window_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    window_count += 1
    now = time.perf_counter()
    elapsed_window = now - window_start

    if not yolo_ran:
        # فقط یک بار YOLO اجرا می‌شود (اینجا در اولین فریم)
        # classes=[2] برای ماشین
        results = model.predict(source=frame, conf=0.3,
                                max_det=2, classes=[0, 3], verbose=False)[0]
        boxes = results.boxes.xyxy.cpu().numpy()
        classes = results.boxes.cls.cpu().numpy().astype(int)

        for box, cls in zip(boxes, classes):
            x1, y1, x2, y2 = map(int, box)
            tracker = cv2.TrackerKCF_create()
            bbox = (x1, y1, x2 - x1, y2 - y1)
            tracker.init(frame, bbox)
            trackers.append(tracker)

            label = model.names.get(cls, 'Unknown')
            labels.append(label)

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX,
                        0.6, (0, 255, 0), 2)

        yolo_ran = True  # علامت می‌زنیم که YOLO اجرا شد
    else:
        # فقط آپدیت Tracker ها
        new_trackers = []
        new_labels = []

        for tracker, label in zip(trackers, labels):
            success, bbox = tracker.update(frame)
            if success:
                new_trackers.append(tracker)
                new_labels.append(label)

                x, y, w, h = map(int, bbox)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX,
                            0.6, (0, 255, 0), 2)

        trackers = new_trackers
        labels = new_labels

    out.write(frame)

    if elapsed_window >= 0.5:
        window_fps = window_count / elapsed_window
        print(
            f"FPS in last {elapsed_window:.2f}s (~0.5s window): {window_fps:.2f}")
        window_start = now
        window_count = 0

total_end = time.perf_counter()
elapsed_total = total_end - total_start
average_fps = frame_count / elapsed_total
print(
    f"Processed {frame_count} frames in {elapsed_total:.2f}s => Average FPS: {average_fps:.2f}")

cap.release()
out.release()
cv2.destroyAllWindows()